In [16]:
import pandas as pd
import os

In [17]:
Desktop = os.path.join(os.path.expanduser("~"), "Desktop")

filepath_root = os.path.join(Desktop, "LBP_Revision")
inclusion_root = os.path.join(filepath_root, "Inclusion_Codes") #inclusion code location
filtered_root = os.path.join(filepath_root, "Filtered_Files") #Filtered data location (Filtered data aggregated all diagnosis by patient)
unfiltered_root = os.path.join(filepath_root, "All_csv_files") #unfiltered data location
combined_root = os.path.join(filepath_root, "LBP_X_Files") #LBP + X filtered location


os.path.exists(filepath_root)

path_inclusion_crit_LBP = os.path.join(inclusion_root,'inclusion_codes_LBP.xlsx')
path_inclusion_crit_DJD = os.path.join(inclusion_root,'inclusion_codes_DJD.xlsx')
path_inclusion_crit_fracture = os.path.join(inclusion_root,'inclusion_codes_fracture.xlsx')
path_inclusion_crit_other = os.path.join(inclusion_root,'inclusion_codes_other.xlsx')
path_inclusion_crit_scoliosis = os.path.join(inclusion_root,'inclusion_codes_scoliosis.xlsx')

In [18]:
def get_cpts_fromfile(filepath=''):
    df_cpt = pd.read_excel(filepath, dtype=str, sheet_name='CPT Codes')
    return df_cpt['CPT Code']       

def get_icds_fromfile(filepath=''):        
    df_icd = pd.read_excel(filepath,dtype=str, sheet_name='ICD Codes')
    return df_icd['ICD Code']

In [19]:
LBP_filename = 'LBP_all'
control_filename = 'control_all'
LBP_all_data = pd.read_csv(unfiltered_root + '/' + LBP_filename + '.csv', encoding = 'utf8',dtype=str)
control_all_data = pd.read_csv(unfiltered_root + '/' + control_filename + '.csv', encoding = 'utf8',dtype=str)

In [20]:
#CPT Code
#LBP only
incl_cpts_LBP = get_cpts_fromfile(path_inclusion_crit_LBP)
incl_cpts_text_LBP = '|'.join(incl_cpts_LBP.str.strip())


#ICD Code
#LBP only
incl_icds_LBP = get_icds_fromfile(path_inclusion_crit_LBP)
incl_icds_text_LBP = '|'.join(incl_icds_LBP.str.strip())


In [21]:
print(LBP_all_data.keys())


Index(['EMPI', 'MGH_MRN', 'Date.prc', 'Date.dia', 'Code_Type.prc', 'Code.prc',
       'Code_Type.dia', 'Code.dia', 'Diagnosis_Name', 'Procedure_Name',
       'Gender_Legal_Sex', 'Age', 'Language', 'Race1', 'Marital_status',
       'Religion', 'Date_Of_Death', 'AgeAtPrc', 'Admit_Date', 'Discharge_Date',
       'Discharge_Disposition', 'diagnosis_list', 'Diagnosis_list_combined',
       'Diagnosis_list_no_NA', 'City', 'State', 'Zip', 'Country',
       'Insurance_1', 'Insurance_2', 'Insurance_3', 'Phone_Code',
       'Physical_Date', 'BMI', 'Height', 'Weight', 'Smoking', 'Alcohol_Use',
       'Helper'],
      dtype='object')


In [22]:
LBP_diagcode = LBP_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
LBP_diagcode['Diagnosis_combined'] = str(0)
LBP_diagcode['LBP?'] = str(0)
print(LBP_diagcode.keys())

LBP_diagcode_reference = LBP_diagcode



Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'LBP?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_32930/3317639091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_32930/3317639091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['LBP?'] = str(0)


In [23]:
LBP_diagcode.loc[0,'Diagnosis_combined'] = LBP_diagcode.loc[0,'Code.dia']
for i in range(1,len(LBP_diagcode)):
    if (LBP_diagcode.loc[i,'EMPI'] == LBP_diagcode.loc[i-1,'EMPI']):
        if LBP_diagcode.loc[i,'Code.dia'] != LBP_diagcode.loc[i-1,'Code.dia']:
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia'] + ', ' + str(LBP_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia']
        continue;

In [24]:
LBP_filtered = LBP_diagcode[LBP_diagcode['Helper'].notna()]
LBP_filtered_save = LBP_filtered

In [25]:
temp_df_LBP = pd.DataFrame
temp_df_LBP = LBP_filtered['Diagnosis_combined'].str.contains(incl_icds_text_LBP, regex=True)

LBP_filtered.index = range(len(LBP_filtered.index))

for i in range(len(temp_df_LBP)):
    if temp_df_LBP.iloc[i] == True:
        LBP_filtered.loc[i,'LBP?'] = 'LBP'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_32930/484879454.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_filtered.loc[i,'LBP?'] = 'LBP'


In [47]:
control_diagcode = control_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
control_diagcode['Diagnosis_combined'] = str(0)
control_diagcode['control?'] = str(0)
print(control_diagcode.keys())

control_diagcode_reference = control_diagcode

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'control?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_32930/206071514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_32930/206071514.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_diagcode['control?'] = str(0)


In [48]:
control_diagcode.loc[0,'Diagnosis_combined'] = control_diagcode.loc[0,'Code.dia']
for i in range(1,len(control_diagcode)):
    if (control_diagcode.loc[i,'EMPI'] == control_diagcode.loc[i-1,'EMPI']):
        if control_diagcode.loc[i,'Code.dia'] != control_diagcode.loc[i-1,'Code.dia']:
            control_diagcode.loc[i,'Diagnosis_combined'] = control_diagcode.loc[i,'Code.dia'] + ', ' + str(control_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            control_diagcode.loc[i,'Diagnosis_combined'] = control_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        control_diagcode.loc[i,'Diagnosis_combined'] = control_diagcode.loc[i,'Code.dia']
        continue;

In [88]:
control_filtered = control_diagcode[control_diagcode['Helper'].notna()]
#control_filtered = control_diagcode
control_filtered_save = control_filtered

In [89]:
for i in range(len(control_filtered)):
    control_filtered.loc[i,'control?'] = 'control'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_32930/3052170580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_filtered.loc[i,'control?'] = 'control'
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_32930/3052170580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_filtered.loc[i,'control?'] = 'control'


In [90]:
LBP_filtered.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?
0,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP
1,100004276,92,Other,21.15,72100,M54.9,useful,M54.9,LBP
2,100006144,78,Black,23.33,72100,G89.29,useful,G89.29,LBP
3,100006348,80,White,27,72040,G89.29,useful,G89.29,LBP
4,100007295,70,White,NaN,74000,G89.29,useful,G89.29,LBP


In [91]:
control_filtered.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,control?
0,100017813,92,White,NaN,99213,L30.9,useful,L30.9,control
7,100069687,93,White,24.36,72040,F43.0,useful,"F43.0, E03.9",control
9,100078641,87,White,NaN,99211,424.1,useful,424.1,control
15,100138213,85,White,26.11,80053,K50.019,useful,"K50.019, D89.9",control
16,100183266,84,White,NaN,99204,G91.2,useful,G91.2,control


In [92]:
LBP_and_control = pd.concat([LBP_filtered,control_filtered])

In [93]:
LBP_and_control.index = range(len(LBP_and_control.index))
#LBP_and_control['Helper'] = str(0)

In [94]:
LBP_and_control.sort_values(by=['EMPI'], inplace=True, ignore_index=True)
LBP_and_control_copy = LBP_and_control
LBP_and_control.head(10)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,control?
0,100000152,81,White,27.02,72100,G89.29,useful,G89.29,LBP,NaN
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP,NaN
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP,NaN
3,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP,NaN
4,100004276,92,Other,21.15,72100,M54.9,useful,M54.9,LBP,NaN
5,100004276,92,Other,17.51,72040,G89.29,useful,G89.29,LBP,NaN
6,100004610,98,White,NaN,72100,S30.0XXA,useful,S30.0XXA,LBP,NaN
7,100006144,78,Black,23.33,72100,G89.29,useful,G89.29,LBP,NaN
8,100006144,78,Black,23.6,72100,M54.9,useful,M54.9,LBP,NaN
9,100006348,80,White,27,72040,G89.29,useful,G89.29,LBP,NaN


In [95]:
duplicateRows = LBP_and_control_copy[LBP_and_control_copy.duplicated(['EMPI'])]
duplicateRows.head(20)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,control?
5,100004276,92,Other,17.51,72040,G89.29,useful,G89.29,LBP,NaN
8,100006144,78,Black,23.6,72100,M54.9,useful,M54.9,LBP,NaN
17,100010159,82,White,18.62,72100,M51.26,useful,M51.26,LBP,NaN
27,100017813,92,White,31.85,12002,I10,useful,"I10, E78.5",NaN,control
28,100017813,92,White,31.28,99214,L23.9,useful,"L23.9, E78.5",NaN,control
29,100017813,92,White,NaN,99213,L30.9,useful,L30.9,NaN,control
32,100019067,77,White,31.14,72100,M54.9,useful,M54.9,LBP,NaN
33,100019067,77,White,27.89,72100,G89.29,useful,"G89.29, M54.9",LBP,NaN
35,100020647,79,Black,23.3,72100,M54.9,useful,M54.9,LBP,NaN
38,100021433,89,Other,33.59,72100,M54.9,useful,M54.9,LBP,NaN


In [96]:
for i in range(1,len(LBP_and_control)):
    if LBP_and_control.loc[i,'EMPI'] == LBP_and_control.loc[i-1,'EMPI']:
        if LBP_and_control.loc[i,'Diagnosis_combined'] != LBP_and_control.loc[i-1,'Diagnosis_combined']:
            LBP_and_control.loc[i,'Diagnosis_combined'] = LBP_and_control.loc[i,'Diagnosis_combined'] + ', ' + str(LBP_and_control.loc[i-1,'Diagnosis_combined'])
        if LBP_and_control.loc[i,'LBP?'] != 'LBP':
            if LBP_and_control.loc[i-1,'LBP?'] == 'LBP':
                LBP_and_control.loc[i,'LBP?'] = 'LBP'
        if LBP_and_control.loc[i,'control?'] != 'control':
            if LBP_and_control.loc[i-1,'control?'] == 'control':
                LBP_and_control.loc[i,'control?'] = 'control'
    else:
        LBP_and_control.loc[i-1,'Helper'] = 'useful'

In [97]:
LBPncontrol_filtered = LBP_and_control.loc[LBP_and_control['Helper'] == 'useful']
LBPncontrol_filtered.reset_index(drop =True, inplace=True)
LBPncontrol_filtered['Both'] = str(0)

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_32930/3102275058.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBPncontrol_filtered['Both'] = str(0)


In [98]:
for i in range(0,len(LBPncontrol_filtered)):
    if LBPncontrol_filtered.loc[i,'LBP?'] == 'LBP':
        if LBPncontrol_filtered.loc[i,'control?'] == 'control':
            LBPncontrol_filtered.loc[i,'Both'] = 'Both'

In [99]:
LBPncontrol_filtered.head(20)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,control?,Both
0,100000152,81,White,27.02,72100,G89.29,useful,G89.29,LBP,NaN,0
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP,NaN,0
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP,NaN,0
3,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP,NaN,0
4,100004276,92,Other,21.15,72100,M54.9,useful,M54.9,LBP,NaN,0
5,100004276,92,Other,17.51,72040,G89.29,useful,"G89.29, M54.9",LBP,NaN,0
6,100004610,98,White,NaN,72100,S30.0XXA,useful,S30.0XXA,LBP,NaN,0
7,100006144,78,Black,23.33,72100,G89.29,useful,G89.29,LBP,NaN,0
8,100006144,78,Black,23.6,72100,M54.9,useful,"M54.9, G89.29",LBP,NaN,0
9,100006348,80,White,27,72040,G89.29,useful,G89.29,LBP,NaN,0


In [100]:
LBP_only = LBPncontrol_filtered.loc[LBPncontrol_filtered['LBP?'] == 'LBP']
LBP_only = LBP_only[LBP_only['Both'] != 'Both']
control_only = LBPncontrol_filtered.loc[LBPncontrol_filtered['control?'] == 'control']
control_only = control_only[control_only['Both'] != 'Both']
LBPcontrol_Both_only = LBPncontrol_filtered.loc[LBPncontrol_filtered['Both'] == 'Both']

In [101]:
LBP_and_control_total = pd.concat([LBP_diagcode,control_diagcode])
LBP_and_control_total.drop_duplicates(subset=['EMPI'],inplace=True)
len(LBP_and_control_total)

4839

In [102]:
print('Number of Total Patients(LBP+control):  ' + str(len(LBP_and_control_total)-1))
print('Number of LBP only:  ' + str(len(LBP_only)))
print('Number of control only:  ' + str(len(control_only)))
print('Number of Both only:  ' + str(len(LBPcontrol_Both_only)))

#LBP_filtered = LBP
#control_filtered = control
#LBP & control filtered = LBPncontrol_filtered


Number of Total Patients(LBP+control):  4838
Number of LBP only:  5220
Number of control only:  2348
Number of Both only:  0


In [103]:
LBP_patients = LBP_filtered.sort_values(by=['EMPI'], ignore_index=True)
LBP_patients.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?
0,100000152,81,White,27.02,72100,G89.29,useful,G89.29,LBP
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP
3,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP
4,100004276,92,Other,17.51,72040,G89.29,useful,G89.29,LBP


In [104]:
control_csv_path = os.path.join(combined_root,'LBP_control_patients.csv')
LBPcontrol_Both_only.to_csv(control_csv_path)

In [105]:
control_only_csv_path = os.path.join(filtered_root,'control_patients.csv')
control_filtered.to_csv(control_only_csv_path)

In [106]:
LBPcontrol_Both_only.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,control?,Both
